# Deploy model

In [1]:
#!pip install -U hopsworks --quiet

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550037
2024-05-06 23:15:17,593 WARNING: using legacy validation callback


Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> Deploy model</span>

In [31]:
%%writefile predict_example.py
import os
import numpy as np
import joblib

class Predict(object):
    def __init__(self):
        """Initializes the serving state, reads a trained model"""
        # Ensure the model path is correct, use an environment variable or direct path
        model_path = os.path.join(os.environ.get("ARTIFACT_FILES_PATH", "/mnt/models"), "xgboost.pkl")
        self.model = joblib.load(model_path)
        print("Initialization Complete")

    def predict(self, inputs):
        """Serves a prediction request using a trained model"""
        # Ensure inputs are correctly shaped
        input_vector = np.array(inputs)
        input_data = input_vector.reshape(1, -1)
        return self.model.predict(input_data).tolist()

Overwriting predict_example.py


In [32]:
import os
dataset_api = project.get_dataset_api()
uploaded_file_path = dataset_api.upload("predict_example.py", "Models", overwrite=True)
predictor_script_path = os.path.join("/Projects", project.name, "Models", "predict_example.py")
print("Path to the predictor script:", predictor_script_path)

Uploading: 100.000%|██████████| 719/719 elapsed<00:01 remaining<00:00

Path to the predictor script: /Projects/BDS24/Models/predict_example.py


## Create the deployment

In [33]:
# Use the model name from the previous notebook.
mr = project.get_model_registry()
model = mr.get_model("BeerRatingPredictionModel", version=1)

# Create and start the deployment
deployment = model.deploy(
    name="beerratingmodeldeployment",  # Give it a descriptive name
    serving_tool="KSERVE",  # Specify the serving tool (e.g., KSERVE)
    script_file=predictor_script_path  # The path to your prediction script
)

print("Deployment: " + deployment.name)
deployment.describe()
deployment.start()

Connected. Call `.close()` to terminate connection gracefully.
Deployment with the same name already exists. Getting existing deployment...
To create a new deployment choose a different name.
Deployment: beerratingmodeldeployment
{
    "artifact_version": 3,
    "batching_configuration": {
        "batching_enabled": false
    },
    "created": "2024-05-06T21:49:09.000Z",
    "creator": "Lasse Hylleberg Laursen",
    "description": null,
    "id": 258051,
    "inference_logging": "NONE",
    "model_framework": "PYTHON",
    "model_name": "BeerRatingPredictionModel",
    "model_path": "/Projects/BDS24/Models/BeerRatingPredictionModel",
    "model_server": "PYTHON",
    "model_version": 1,
    "name": "beerratingmodeldeployment",
    "predictor": "predict_example.py",
    "predictor_resources": {
        "limits": {
            "cores": 0.5,
            "gpus": 0,
            "memory": 1024
        },
        "requests": {
            "cores": 0.2,
            "gpus": 0,
            "mem